In [1]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import itertools
import functools
import importlib
import os
from IPython.display import Image

from nl_00_hmdb_pre import main_loop_0
from sklearn.decomposition import PCA
import umap

from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde

import warnings

def action_with_warnings():
    warnings.warn("should not appear")

In [ ]:
# Regression:
from nl_03_filter_model_score_reg import filter_split_model_score

In [ ]:
# Binary classification:
from nl_03_filter_model_score import filter_split_model_score

### This script runs the neutral loss scripts for MALDI IMS data

Steps:
1. Generation of core metabolome database.  See:
    PycharmProjects/core_metabolome/core_metabolome_db.ipynb
2. 

1. nl_00 Calculates Mordred descriptors and FP4 fingerprints as bits.
2. nl_01 Filter HMDB_db for observed and parses METASPACE output.
3. nl_02 Joins HMDB_db and METASPACE output.
4. nl_03 Filters and searchs with direct and machine learning models.

### To do:
5. nl_04 deep learning: executing on toy data, change to regression

### Observations:
1. 2912/3333 datasets a) did not error and b) had at least one id.  
2. 2543/2868 datasets w/10 or more ID's.  Obs: 557,508/559,002
3. 644,094 values identifications, the dataset with the most identifications had 5,820 and mean was 221 (median 131).  Re calculate...
4. The data did not pass the Shapiro-Wilk test for normality, and nonparametrical statistics should be used.

# Off-line steps:
### Generation of core metabolome database.
http://localhost:8888/notebooks/PycharmProjects/core_metabolome/core_metabolome_db.ipynb
### Identification of good quality datasets.
http://localhost:8888/notebooks/PycharmProjects/neutral_loss/good_nl_reports/high_quality_data_investigations.ipynb
### Good datasets were researched from the beta server.  Run 1x w/ "reprocess = True" and "reprocess_not_downloading = True".  Run again w/False.
http://localhost:8888/notebooks/PycharmProjects/neutral_loss/good_nl_reports/Beta_server_neutral_losses_mass_search.ipynb

In [3]:
### Off-line variables:
core_metabolome = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/core_metabolome_v2.pickle'
good_ds_ids = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/good_ds_2020_Feb_25.txt'
beta_variables = {'NEUTRAL_LOSSES': ['-H2O'], 'MAX_FDR': 0.5, 'molDBs': 'core_metabolome_v2'}
path_to_reports = 'tbd'

In [4]:
### Off-line steps:

# Generation of core metabolome database.  See:
'http://localhost:8888/notebooks/PycharmProjects/core_metabolome/core_metabolome_db.ipynb'
core_metabolome = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/core_metabolome_v2.pickle'

# Identification of good quality datasets.
'http://localhost:8888/notebooks/PycharmProjects/neutral_loss/good_nl_reports/high_quality_data_investigations.ipynb'
good_ds_ids = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/good_ds_2020_Feb_25.txt'

# Good datasets were researched from the beta server.  Run 1x w/ "reprocess = True" and
# "reprocess_not_downloading = True".  Run again w/False.  See:
'http://localhost:8888/notebooks/PycharmProjects/neutral_loss/good_nl_reports/Beta_server_neutral_losses_mass_search.ipynb'
beta_variables = {'NEUTRAL_LOSSES': ['-H2O'], 'MAX_FDR': 0.5, 'molDBs': 'core_metabolome_v2'}
path_to_reports = 'tbd'

In [5]:
# nl_00: Generates FP4 descriptors ('bits') and Mordred descriptors. 
# 900 seconds to reprocess everything on 13k. 15s without reprocessing.

db_out_0 = 'databases/core_metabolome_out.pickle'
db_in_0 = core_metabolome
dfs = main_loop_0(db_out_0, db_in_0, True, False)

db_df = dfs[0]
bits_df = dfs[1]
mord_norm_df = dfs[2]

Elapsed time:

16.36795711517334

Executed without error

databases/core_metabolome_out.pickle


In [8]:
# Merge pickles for each dataset.
path_to_reports = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/reports'

master_df = pd.DataFrame()
counter = 0

for root, dirs, files in os.walk(path_to_reports):
    for file in files:
        if file.endswith(".pickle"):
            if counter % 50 == 0:
                print(counter)
            counter +=1
            current_df = pd.read_pickle(root + '/' + file)
            master_df = pd.concat([master_df, current_df], sort=True)

0
50
100
150
200
250
300


In [14]:
ds_id_meta

,datasetId,group,analyzer,polarity,FDR10-v4
10,2018-10-12_09h21m15s,U Copenhagen,Orbitrap,negative,129
11,2018-10-12_09h25m16s,U Copenhagen,Orbitrap,negative,135
12,2018-10-16_09h53m21s,U Copenhagen,Orbitrap,negative,115
15,2018-10-31_10h26m49s,U Copenhagen,Orbitrap,negative,164
20,2018-11-08_15h29m23s,U Copenhagen,Orbitrap,negative,101
...,...,...,...,...,...
3213,2017-01-19_18h32m25s,♡EMBL♡,Orbitrap,negative,112
3214,2017-01-25_17h34m34s,♡EMBL♡,Orbitrap,negative,581
3217,2017-01-20_17h52m13s,♡EMBL♡,Orbitrap,negative,216
3225,2019-11-28_11h24m47s,♡EMBL♡,Orbitrap,positive,179


In [21]:
# Water only.  Edit for cases where neutral losses other than water under investigation.
terse_headers_0 = ['formula', 'adduct', 'ds_id', 'has_no_loss', 'has_H2O',
                   'msm', 'fdr', 'off_sample', 'hmdb_ids', 'intensity_avg',
                   'msm_H2O', 'fdr_H2O', 'off_sample_H2O', 'intensity_avg_H2O',
                   'colocalization_H2O', 'loss_intensity_ratio_H2O', 'loss_intensity_share_H2O',
                  'ion', 'ion_H2O', 'ion_formula', 'ion_formula_H2O']

master_df = master_df[terse_headers_0].copy(deep=True)
ds_id_meta = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/ds_id_meta.pickle')
master_df = master_df.merge(ds_id_meta, left_on='ds_id', right_on= 'datasetId', how='left')
master_df.drop(columns=['datasetId']).to_pickle('high_quality.pickle')

In [7]:
good_ds_ids

'/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/good_ds_2020_Feb_25.txt'

Exploratory data analysis:
http://localhost:8888/notebooks/PycharmProjects/neutral_loss/Exploratory_data_analysis.ipynb

In [ ]:
# Loads pickles from previous step
# For QC, don't need to run for reprocess.
input_df = pd.read_pickle('high_quality.pickle')
hmdb_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/core_metabolome_out.pickle') # 0.1 Gb
mord_norm_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/mord_norm_df.pickle')  # 0.8 Gb
bits_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/bits_df.pickle')  #0.2 Gb

In [5]:
### nl_01 to nl_03 need to be updated after exploratory data analysis is finished!!!

In [ ]:
# nl_01: Parses METASPACE output and filters HMDB for observed
%run nl_01_preprocess.py -m all_public_data.pickle -p all_public_hmdb.pickle -is_H2O True

In [ ]:
# Loads pickles from previous step
# For QC, don't need to run for reprocess.
output_1 = pd.read_pickle('all_public_data_output_01.pickle')
hmdb_1 = pd.read_pickle('all_public_data_hmdb_01.pickle')

In [ ]:
# nl_02: Joins METASPACE output and observed HMDB
%run nl_02_join.py --o all_public_data_output_01.pickle --h all_public_data_hmdb_01.pickle

In [ ]:
# Load X's if running machine learning
join_df_path = 'all_public_data_output_02.pickle' 
join_df = pd.read_pickle(join_df_path)  # 0.5 Gb
unique_hmdbs = list(join_df.hmdb_ids.unique())

mord_df = pd.read_pickle('mord_norm_hmdb_df.pickle') # 0.2 Gb
mord_df = mord_df[mord_df.hmdb_ids.isin(unique_hmdbs)].copy(deep=True)

bits_df = pd.read_pickle('bits_hmdb_df.pickle') # 0.8 Gb
bits_df = bits_df[bits_df.hmdb_ids.isin(unique_hmdbs)].copy(deep=True)

In [ ]:
# Can use this to filter on inferred vaccuum maldi vs ap maldi
# ap 1921, vav 1228, all 3248
beta_METASPACE = pd.read_csv('Metaspace_beta_2020_Feb.tsv', sep='\t')
inf_source = beta_METASPACE[['datasetId', 'analyzer']].copy(deep=True)

all_ds = list(inf_source.datasetId)

inf_source = inf_source[(inf_source.analyzer == 'FTICR') |
                       (inf_source.analyzer == 'Orbitrap')].copy(deep=True)
ap_ds = list(inf_source[inf_source.analyzer == 'Orbitrap'].datasetId)
vac_ds = list(inf_source[inf_source.analyzer == 'FTICR'].datasetId)

In [ ]:
# nl_03: Filtering and machine learning models
datasets = ['all_ds', 'ap_ds', 'vac_ds']
ds_dict = {'all_ds': all_ds, 'ap_ds': ap_ds, 'vac_ds': vac_ds}
dataset = [datasets[1]]
target = ['H2O'] # ['H2O']
polarity = [1] # {1:positive, -1:negative] 
fdrs = [0.20] # [0.20, 0.10, 0.05]
colocalizations = [0.50] # [0.00, 0.50, 0.75]
min_n_obs = [10] # [1, 4, 10]
one_id_only = [False] # [True, False]

# Outputs (y), and weights (w):
global_ys = ['n_loss_only_H2O', 'n_loss_wparent_H2O', 'n_loss_all_H2O']
y = ['n_loss_wparent_H2O']
w = ['weight']
w_norms = [False, '10_y_bins', 'n_obs', 'isobar',
          '10_y_bins_W_n_obs', '10_y_bins_W_isobar', 'n_obs_W_isobar',
          '10_y_bins_W_n_obs_W_isobar']
w_norm = ['n_obs']

# Class of model:
models = ['direct', 'ml', 'deepchem']
model = [models[1]]

# Split.  If true, any formula only appears in train/test/validate.
single_fold_group = True

filter_model_df = pd.DataFrame(columns=['target', 'polarity', 'fdr', 'coloc', 'min_n_obs'
                                       'model', 'submodel', 'params', 'one_id_only',
                                        'X', 'y', 'w'])
# Specific inputs (X) for each model:
if model[0] is 'direct':
    submodel = ['']
    params = ['']
    Xs = ['trues', 'falses', 'rando', 'H2O_Present', 'n_loss_wparent_H2O']
    X = Xs                           
elif model[0] is 'ml':
    submodels = ['random_forest', 'XGBoost', 'SVM']
    submodel = [submodels[1]]
    params = ['']
    Xs = ['mord_norm', 'bits'] 
    X = ['mord_norm']
elif model[0] is 'deepchem':
    submodels = [('GraphConvModel', 'GraphConv'), 
              ('WeaveModel', 'Weave'), 
              ('MPNNModel','Weave') ] 
    submodel = [submodels[0]]    
    params = ['']
    Xs = ['Molecule', 'Smiles']
    X = [Xs[0]]          
    
model_list = list(itertools.product(*[target, polarity, fdrs, colocalizations, min_n_obs,
                                      model, submodel, params, one_id_only,
                                      X, y, w, w_norm, dataset]))
print()
filter_param_df = pd.DataFrame(model_list, columns=['target', 'polarity', 'fdr', 'coloc', 'min_n_obs',
                                                    'model', 'submodel', 'params', 'one_id_only',
                                                    'X', 'y', 'w', 'w_norm', 'dataset'])

filter_param_df

In [ ]:
model_results = filter_split_model_score(filter_param_df, join_df_path, mord_df, bits_df,
                                         single_fold_group, ds_dict[dataset[0]])

In [ ]:
model_results.round(3)

In [ ]:
model_results.sort_values(by=['rmse_test_w']).round(3)

In [ ]:
# Regression results:
all_model_results = pd.read_pickle('all_model_results.pickle')
all_model_results = pd.concat([all_model_results, model_results])
all_model_results.to_pickle('all_model_results.pickle')
all_model_results.round(3)

In [ ]:
all_model_results.sort_values(by=['rmse_test']).round(3)

In [ ]:
temp = dict(all_model_results[(all_model_results.index == 90) & 
                  (all_model_results.X == 'mord_norm')].round(2))

In [ ]:
temp